<a href="https://colab.research.google.com/github/mzk8888/AI/blob/main/qa4u3_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

今回はシミュレータメインで行きます。
大量に計算を複数回実行しながら検討することが多いためです。
さらにJij Inc.が用意したjijmodelingを利用した最適化問題のプログラムについて紹介します。まずはjijmodeling_transpilerをインストールしましょう。

In [ ]:
pip install jijmodeling_transpiler

その後にいつものopenjijをインストールします。

In [ ]:
pip install openjij

インストールしたら必要なライブラリをして以下のようにimportしておきましょう。
samplerもSASamplerを利用するということで準備しておきます。

In [ ]:
import jijmodeling as jm
import jijmodeling_transpiler as jmt
from openjij import SASampler
import numpy as np
sampler = SASampler()

## 数分割問題

2つのグループに与えられた数値を均等に割ることを考えます。
これは例えば戦力とか財産とか色々なものを想定することができます。

In [ ]:
N = 20

合計で20個の候補の数値があるとしましょう。
乱数によって決まった出鱈目な数値があるとします。
これらをAグループなら1、Bグループなら0となるような$x_i$を用意します。

In [ ]:
x = jm.BinaryVar('x', shape=(N, ))
i = jm.Element('i', belong_to=(0, N))

jijmodelingでは変数BinaryVarと添え字Elementを宣言して用意します。

In [ ]:
W = jm.Placeholder('W',ndim = 1)

出鱈目な数字を入れておく場所としてPlaceholderというものも用意します。
ここに後から数字を入れるイメージです。

In [ ]:
problem = jm.Problem('Number partition', sense=jm.ProblemSense.MINIMIZE)
problem += (jm.sum(i,W[i]*x[i]) - jm.sum(i,W[i]*(1-x[i])))**2

次は最適化問題の設定です。
QUBOを直接書くのでも良いですが、上記のように数式ベースで書くこともできます。
シグマ記号の代わりに和を取るのだよ、ということでsum(i,...)など、
和の添え字と内容を書きます。
それをproblemに追加して最適化問題を定義します。

In [ ]:
problem

どんな数式の最適化問題を解くことになったのか、problemから見ることができるのも嬉しい機能です。
さてこの数式にWのところに具体的な数値を入れることにしましょう。

In [ ]:
Wvec = np.random.rand(N)
instance_data = {'W': Wvec}

これで準備完了です。
おまじない的ですがjijmodelingで準備された最適化問題をquboに直してもらいます。

In [ ]:
compiled_model = jmt.core.compile_model(problem, instance_data, {})
pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
qubo, const = pubo_builder.get_qubo_dict()

これで出力されたquboを利用すれば最適化問題をシミュレーテッドアニーリングないし量子アニーリング（しかも他のソルバーも利用できる）で解くことができます。

In [ ]:
num_reads = 10
sampler = SASampler()
sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)

もちろん結果を見るのも今まで通りで差し支えありません。

In [ ]:
sampleset.record

ただこれもどこが0でどこが1なのか抽出できた方が便利なことがあります。
それも以下のようなコードで見やすくすることができます。

In [ ]:
result = jmt.core.pubo.decode_from_openjij(sampleset, pubo_builder, compiled_model)

ここで得られたresultからresult.record.solution["x"]とすればどこが1になっているのかが一目瞭然です。["x"]の後に[0]なり[1]と打つと、最もエネルギーが低い結果から順に得ることができます（ソート済み）

In [ ]:
result.record.solution["x"][0][0][0]

順番になっているので0番目の答えを抽出すると、一番エネルギーの低い結果が出てきます。さらに[0][0]とすると何番目のスピンが1として立ち上がっているのかがわかります。

## グループ分けの最適化

数分割問題を参考にして、グループに所属しているというのを$1$で、所属していないというのを$0$で表すものとします。
$x_{ij}$でi番目の人をj番目のグループに入れるかどうかを決めるとしましょう。
同じグループになる人同士では$C_{ii'}$という相性を示す係数がかかり、相性ができるだけ最大化されるようにしましょう。

In [ ]:
N = 20
M = 5

20人を5人のチームに分けます。
Jij modelingによる記法で書いてみましょう。

In [ ]:
x = jm.BinaryVar('x', shape=(N, M))
i1 = jm.Element('i1', belong_to=(0, N))
i2 = jm.Element('i2', belong_to=(0, N))
j = jm.Element('j', belong_to=(0, M))

あとあとわかることですが添え字を複数利用するので、i1とi2としました。

次に係数を用意します。相性を示す係数を以下のようにPlaceholder（後から数値を入れる場所）として用意します。

In [ ]:
C = jm.Placeholder('C',ndim = 2)

今回は添え字が2つあるのでndim=2というオプションとなります。

ちなみにjijmodelingでは最小化だけではなく最大化も式をいじることなく指定することができます。
相性を最大化することにしましょう。

In [ ]:
problem = jm.Problem('Grouping', sense=jm.ProblemSense.MAXIMIZE)
problem += jm.sum([i1,i2,j], C[i1,i2]*x[i1,j]*x[i2,j])
problem += jm.Constraint("one group",  jm.sum(j,x[i1,j]) == 1, forall=i1)

ここで制約条件というものを追加しました。Constraintです。
jについての和ですから、グループ全てで和を取ると、どれか1つのグループにいるように、という指示をしています。
これが各人に適用されるのでforall=i1というオプションをつけています。
出来上がったモデルの数式を眺めてみましょう。

In [ ]:
problem

さて適当な乱数で相性を計算して入れてみましょう。

In [ ]:
Cmat = np.random.randn(N*N).reshape(N,N)
instance_data = {'C': Cmat}

以降は全く一緒です。
ただ制約条件を入れた場合には、multipliersで係数を指定しておく必要があります。
これは罰金法と呼ばれる前回も紹介した方法により、制約条件を満たしている場合を最適解とするように誘導する方法を利用しているためです。
その係数が大きければ大きいほど制約条件に忠実になります。

In [ ]:
compiled_model = jmt.core.compile_model(problem, instance_data, {})
pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
qubo, const = pubo_builder.get_qubo_dict(multipliers={"one group": 2.0})

出来上がったquboを用いて最適化を実行します。

In [ ]:
sampler = SASampler()
sampleset = sampler.sample_qubo(qubo, num_reads=100)

ここで重要なのが制約条件を満たした答えを抽出することができることです。
resultからfeasibleとしてみましょう。

In [ ]:
result = jmt.core.pubo.decode_from_openjij(sampleset, pubo_builder, compiled_model)
# get feasible samples from sampleset
feasible_samples = result.feasible()

これで実行可能解（制約条件を満たした解）を取り出すことができました。

In [ ]:
feasible_samples.record.solution["x"][0][0][1]

何番目のグループに所属するのかが一目瞭然の結果を得ることができます。

ここで同じ番号になっているグループの中では相性が良い人同士を集めています。
相性の数値はマイナスもあるのであるのでそういう人同士はなるたけ入らないように組み合わせています。

## 非負値行列分解

次にたくさんの顔画像データから人類の顔の特徴を学ぶことに挑戦してみましょう。
行列分解という方法を利用します。

まずは顔のデータセット
https://web.archive.org/web/20180208155206/http://www.ai.mit.edu/courses/6.899/lectures/faces.tar.gz
からDLして前回と同様の手続きでGoogle colabにuploadしましょう。

In [ ]:
from google.colab import files
uploaded = files.upload()

その後解凍をします。

In [ ]:
!tar -zxvf faces.tar.gz

さらにface.train.tar.gzがあるのでこちらも解凍をしましょう。そうするとたくさんの顔画像が入ったtrain/non-faceデータが得られます。

In [ ]:
!tar -zxvf face.train.tar.gz

前回同様にフォルダにあるファイル名を全て取得します。

In [ ]:
import glob
files = glob.glob(r"/content/train/face/*")

ひとつ取り出してみてみましょう。

画像を読み込むのにopencvを利用します。

In [ ]:
import cv2
im = cv2.imread(files[101])
mat = im[:,:,0]

画像の表示にはimshowで実行します。

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(mat)
plt.show()

何かしら切り取られた顔データが見られるかと思います。
このような様々な人の顔が含まれるデータセットから
共通するパーツを抜き出そうというのがここでやりたいことです。
さらに足し算のみでこれを記述しましょう、という非負値行列分解という方法があります。
そこからさらに0と1でどの顔のパーツを使うか使わないかを調べられるように改良したものを考えます。

まずは１枚１枚データを格納しておいたものをydataとします。

In [ ]:
ylist = []
for file in files:
  im = cv2.imread(file)
  y = im[:,:,0].flatten()
  ylist.append(y)

ydata = np.array(ylist)

このydataをうまく説明する行列の２組を見つけます。
1つは辞書的な行列で色々な絵の要素を持つものです。
この要素を組み合わせて１枚の画像がうまく作り出せると考えます。
１枚１枚のデータに対して、0と1を合わせて
全部のデータに対して共通する画像のパーツを見出せば良いという問題です。

まず全画像のデータ数をMとします。
これを1つ1つ見せて、0と1で分解します。
Nは画像のデータのサイズを示します。
K=20個として20個の成分で分解することを考えます。
まず20個の特徴ベクトルを用意しましょう。1つ1つはNの画像データを表現可能で、それを20個のうちいくつか組み合わせて実際の画像とすることを考えます。



In [ ]:
M = len(ylist)
N = len(y)
K = 10
x = jm.BinaryVar('x', shape=(K,))
k1 = jm.Element('k1', belong_to=(0, K))
k2 = jm.Element('k2', belong_to=(0, K))
a = jm.Placeholder('a',ndim = 1)
Q = jm.Placeholder('Q',ndim = 2)
lam = jm.Placeholder('lam',ndim = 0)

aとQというパラメータを持つ以下のようなコスト関数を用意します。
ここでaは類似度、Qは特徴ベクトル同士の重なりを減らすようにする効果があります。
つまり利用する特徴ベクトルの間で直交化するようにしています。

In [ ]:
problem = jm.Problem('NMF_face', sense=jm.ProblemSense.MINIMIZE)
sum1 = -2*jm.sum(k1,a[k1]*x[k1])
sum2 = jm.sum([k1,k2],Q[k1,k2]*x[k1]*x[k2])
sum3 = lam*jm.sum(k1,x[k1])
problem += sum1 + sum2 + sum3

試しにでたらめな特徴ベクトルをまずは用意して、顔画像１枚を使って
各係数を計算してみます。

In [ ]:
W = np.random.rand(N*K).reshape(N,K)
d = 0
yvec = ylist[d]
Qmat = np.dot(W.T,W)
avec = np.dot(yvec,W)
lam = 1.0
instance_data = {'Q': Qmat , "a": avec, "lam": lam}

Qmatでは特徴ベクトル間の積をとって直交度合いを探ります。
またavecでは画像と特徴ベクトルの積を調べて、どの特徴ベクトルを使ったらうまく画像を再現できそうなのかを調べています。

In [ ]:
compiled_model = jmt.core.compile_model(problem, instance_data, {})
pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
qubo, const = pubo_builder.get_qubo_dict(multipliers={})

この最適化問題を解いて0と1、つまりどの特徴ベクトルを利用するのかを選択してもらいましょう。

In [ ]:
num_reads = 10
sampler = SASampler()
sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)

この結果を利用してボルツマンマシンの際と同じように、
サンプリングした結果の傾向を見ておきます。
前回と同様に以下のような計算をしておきます。

In [ ]:
def calc_grad(sampleset,num_reads=num_reads):
  qubo_model = np.zeros(K**2).reshape(K,K)
  for k in range(num_reads):
    qubo_model = qubo_model + np.outer(sampleset.record[k][0],sampleset.record[k][0])/num_reads
  return qubo_model

この傾向から、
まずyvecのどれを参考にするのかをnp.diag(qubo_model)と積をとって調べます。
その際に複数の画像の結果について

In [ ]:
eta = 0.1
Tall = 20

for t in range(Tall):
  for d in range(M//100):
    yvec = ylist[d]
    Qmat = np.dot(W.T,W)
    avec = np.dot(yvec,W)
    lam = 1.0
    instance_data = {'Q': Qmat , "a": avec, "lam": lam}
    compiled_model = jmt.core.compile_model(problem, instance_data, {})
    pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
    qubo, const = pubo_builder.get_qubo_dict(multipliers={})
    sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)
    result = jmt.core.pubo.decode_from_openjij(sampleset, pubo_builder, compiled_model)
    face_list = result.record.solution["x"][0][0][0]
    qubo_model = calc_grad(sampleset)
    plt.figure(figsize=(18, 4))
    plt.subplot(1,len(face_list)+2,1)
    plt.imshow(yvec.reshape(19,19))
    face_sol = np.zeros(N).reshape(19,19)
    for k in range(len(face_list)):
      plt.subplot(1,len(face_list)+2,k+2)
      plt.imshow(W[:,face_list[k]].reshape(19,19))
      face_sol = face_sol + W[:,face_list[k]].reshape(19,19)
    plt.subplot(1,len(face_list)+2,len(face_list)+2)
    plt.imshow(face_sol.reshape(19,19))
    plt.show()
    W = W + eta*(np.outer(yvec.reshape(N,1),np.diag(qubo_model).reshape(1,K)) - np.dot(W,qubo_model))
    W = W.clip(min=0)

できあがったようですね。特徴ベクトルの1本を取り出して見ましょう。
そうすると人類の顔の傾向から代表的な20本のベクトルが得られます。

人類の顔はこれをいくつか選び出して組み合わせてできあがっているということになります。
顔画像は一見面白く良い結果が出てきそうですが、実は危険です。
実際は顔の画像は向きや角度、照明などそのまま利用するには基準がずれたものが多いので難しいのが実際です。
フォーマットができるだけ揃ったデータを利用すると良いでしょう。
そういう意味では音楽などはフォーマットが揃った良いデータだと思います。
つまり良くあるフレーズを抜き出すということができるというわけです。

## 音楽の要素抽出

前回のNESのデータを使って、音楽の典型的な要素を抽出するということもできそうですね。
前回のおまじないをここでまずは実行していきましょう。

In [ ]:
!pip install music21

In [ ]:
!apt-get install musescore

In [ ]:
!apt-get install xvfb

In [ ]:
!sh -e /etc/init.d/x11-common start

In [ ]:
import os
os.putenv('DISPLAY', ':99.0')

In [ ]:
!start-stop-daemon --start --pidfile /var/run/xvfb.pid --make-pidfile --background --exec /usr/bin/Xvfb -- :99 -screen 0 1024x768x24 -ac +extension GLX +render -noreset

In [ ]:
from music21 import *
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/mscore'
us['musicxmlPath'] = '/usr/bin/mscore'
us['directoryScratch'] = '/tmp'

それではここで前回のNESの音楽ファイルをまたアップロードしましょう。
https://github.com/chrisdonahue/nesmdb ダウンロードしたのち、個々のGoogle colab経由でアップロードして利用しましょう。 利用するのはSeparated Score Formatです。

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!tar -zxvf nesmdb24_seprsco.tar.gz

In [ ]:
files = glob.glob(r"/content/nesmdb24_seprsco/train/*")

length = 8

import pickle
data_dict = {}
for file in files:
  with open(file, 'rb') as f:
    rate, nsamps, seprsco = pickle.load(f)
    if seprsco.shape[0] > length:
      data_dict[file] = [rate, nsamps, seprsco]
      print(file, nsamps, seprsco.shape)

In [ ]:
def create_roll(piano_roll):
  ans = np.zeros(piano_roll.shape[0])
  itemp0 = np.where(piano_roll==0)
  itemp1 = np.where(piano_roll!=0)
  ans[itemp0] = 0 #0で休符
  ans[itemp1] = np.mod(piano_roll[itemp1]-1,12) + 1 #1-12で音階
  return ans

ここまでは共通したもので、
ここから少し変更していきます。
まずnum_dataで指定した数分の音楽のデータを手に入れます。

In [ ]:
key_list = list(data_dict.keys())
num_data = 100
random_nums = np.random.randint(0,len(key_list),num_data)

ylist = []
for k in range(num_data):
  piano_rolls = data_dict[key_list[random_nums[k]]][2]
  music_length = len(piano_rolls[:,0])
  mstart = np.random.randint(0,music_length-length)
  yvec = np.zeros([length,4])
  for m in range(4):
    piano_roll = piano_rolls[mstart:mstart+length,m]
    yvec[:,m] = create_roll(piano_roll)
  ylist.append(yvec.flatten())

これでylistには音階と休符の0-12までの整数値が4パート分並んだデータを入れてあります。
これを同じように行列分解すれば共通する基本旋律が得られると考えられます。
（それ自体に意味があるというより足し算をして組み合わせると意味のある音楽になりやすい）

途中でどんな楽譜が登場するかも気になるので、結果得られた旋律を楽譜表示をするコードを書きましょう。


In [ ]:
note_dict = {}
note_dict[0] = 'r'
note_dict[1] = 'c'
note_dict[2] = 'c#'
note_dict[3] = 'd'
note_dict[4] = 'd#'
note_dict[5] = 'e'
note_dict[6] = 'f'
note_dict[7] = 'f#'
note_dict[8] = 'g'
note_dict[9] = 'g#'
note_dict[10] = 'a'
note_dict[11] = 'a#'
note_dict[12] = 'b'

前回利用した数値を音符に変える対応表を使います。

In [ ]:
def score_create(yvec):
  gen_score = 'tinyNotation: 4/4'
  ymat = yvec.reshape(length,4)
  for t in range(length):
    gen_score += ' ' + note_dict[int(ymat[t,0].clip(max=12))]
  cp = converter.parse(gen_score)
  return cp

整数値でないと楽譜に変換できないのでintを利用しています。主旋律だけまずはやってみましょう。


In [ ]:
M = len(ylist)
N = length*4
K = 10
x = jm.BinaryVar('x', shape=(K,))
k1 = jm.Element('k1', belong_to=(0, K))
k2 = jm.Element('k2', belong_to=(0, K))
a = jm.Placeholder('a',ndim = 1)
Q = jm.Placeholder('Q',ndim = 2)
lam = jm.Placeholder('lam',ndim = 0)

Nはlength*4にします。4パート分一気に学習することができます。
（表示は主旋律だけにします）

In [ ]:
problem = jm.Problem('NMF_music', sense=jm.ProblemSense.MINIMIZE)
sum1 = -2*jm.sum(k1,a[k1]*x[k1])
sum2 = jm.sum([k1,k2],Q[k1,k2]*x[k1]*x[k2])
sum3 = lam*jm.sum(k1,x[k1])
problem += sum1 + sum2 + sum3

In [ ]:
W = 12*np.random.rand(N*K).reshape(N,K)
eta = 0.1
Tall = 20

for t in range(Tall):
  for d in range(M):
    yvec = ylist[d]
    Qmat = np.dot(W.T,W)
    avec = np.dot(yvec,W)
    lam = 1.0
    instance_data = {'Q': Qmat , "a": avec, "lam": lam}
    compiled_model = jmt.core.compile_model(problem, instance_data, {})
    pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
    qubo, const = pubo_builder.get_qubo_dict(multipliers={})
    sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)
    result = jmt.core.pubo.decode_from_openjij(sampleset, pubo_builder, compiled_model)
    score_list = result.record.solution["x"][0][0][0]
    qubo_model = calc_grad(sampleset)
    print("元の音源")
    cpy = score_create(yvec)
    cpy.show()
    score_sol = np.zeros(N)
    print("音源要素候補")
    for k in range(len(score_list)):
      cptemp = score_create(W[:,score_list[k]])
      cptemp.show()
      score_sol = score_sol + W[:,score_list[k]]
    print("再現")
    cp = score_create(score_sol)
    cp.show()
    W = W + eta*(np.outer(yvec.reshape(N,1),np.diag(qubo_model).reshape(1,K)) - np.dot(W,qubo_model))
    W = W.clip(min=0,max=12)

## 映画の推薦

MovieLensのデータセットを利用します。
https://grouplens.org/datasets/movielens/
このデータセットでは（教育用：87,585個の映画について200,948のユーザーによる）映画の５段階評価のデータが用意されています。
これに基づきユーザーのパターンを習得し、未知のユーザーに対しても部分的情報からそのパターンを見つけ出して、推薦をすることなどができます。
但しここでは顔のデータとは異なり、欠損（未評価）データがあることにご注意を。


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip ml-latest-small.zip

csvファイルで用意されたデータをpandasで読み込む。

In [ ]:
import pandas as pd
df = pd.read_csv('ml-latest-small/ratings.csv')

In [ ]:
df

ここで先ほど$y$としていたデータに映画ごとのレーティングを入れることにしましょう。

In [ ]:
N = df["movieId"].to_numpy().max()
M = df["userId"].to_numpy().max()
Ymat = np.zeros(N*M).reshape(N,M)

for k in range(len(df)):
  i = df["movieId"][k] - 1
  j = df["userId"][k] - 1
  Ymat[i,j] = df.iloc[k]["rating"]

これは映画の種類がめちゃくちゃ多いので、非常に縦長のデータになる。
しかしユーザーごとに量子アニーリングマシンに投入してその結果に基づき
Wという特徴を示す行列を更新していくことができる。

In [ ]:
K = 20
x = jm.BinaryVar('x', shape=(K,))
k1 = jm.Element('k1', belong_to=(0, K))
k2 = jm.Element('k2', belong_to=(0, K))
a = jm.Placeholder('a',ndim = 1)
Q = jm.Placeholder('Q',ndim = 2)
lam = jm.Placeholder('lam',ndim = 0)

量子アニーリングマシンに投げるところなどは全て同じ形なのでそのまま採用できる。
そういう場合はjijmodelingでは係数だけ変更すれば良いので便利である。

In [ ]:
problem = jm.Problem('NMF_movie', sense=jm.ProblemSense.MINIMIZE)
sum1 = -2*jm.sum(k1,a[k1]*x[k1])
sum2 = jm.sum([k1,k2],Q[k1,k2]*x[k1]*x[k2])
sum3 = lam*jm.sum(k1,x[k1])
problem += sum1 + sum2 + sum3

但しこの映画のデータの場合には未観測の部分があるので、それを0ではなく足し算に入れないという処理をする必要がある。

In [ ]:
W = 5*np.random.rand(N*K).reshape(N,K)
d = 0
yvec = Ymat[:,d]
mask = np.where(yvec!=0)[0]
Wmask = W[mask,:]
ymask = yvec[mask]
Qmat = np.dot(Wmask.T,Wmask)
avec = np.dot(ymask,Wmask)
instance_data = {'Q': Qmat , "a": avec, "lam": 1.0}

以降は全く同じ。

In [ ]:
compiled_model = jmt.core.compile_model(problem, instance_data, {})
pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
qubo, const = pubo_builder.get_qubo_dict(multipliers={})

In [ ]:
num_reads = 10
sampler = SASampler()
sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)

In [ ]:
eta = 0.1
Tall = 20

for t in range(Tall):
  for d in range(M):
    yvec = Ymat[:,d]
    mask = np.where(yvec!=0)[0]
    Wmask = W[mask,:]
    ymask = yvec[mask]
    Qmat = np.dot(Wmask.T,Wmask)
    avec = np.dot(ymask,Wmask)
    lam = 1.0
    instance_data = {'Q': Qmat , "a": avec, "lam": lam}
    compiled_model = jmt.core.compile_model(problem, instance_data, {})
    pubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_model=compiled_model)
    qubo, const = pubo_builder.get_qubo_dict(multipliers={})
    sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)
    result = jmt.core.pubo.decode_from_openjij(sampleset, pubo_builder, compiled_model)
    qubo_model = calc_grad(sampleset)
    plot_list = result.record.solution["x"][0][0][0]
    plt.plot(yvec[mask])
    sol_temp = np.zeros(len(mask))
    for k in range(len(plot_list)):
      plt.plot(Wmask[:,plot_list[k]],alpha=0.2)
      sol_temp = sol_temp + Wmask[:,plot_list[k]]
    plt.plot(sol_temp.astype(int))
    plt.ylim(0,5.1)
    plt.show()
    W[mask,:] = W[mask,:] + eta*(np.outer(ymask.reshape(len(ymask),1),np.diag(qubo_model).reshape(1,K)) - np.dot(W[mask,:],qubo_model))
    W = W.clip(min=0,max=5)